<a href="https://colab.research.google.com/github/Tonoyama/DL-Study/blob/master/original_data_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 自作データセットを CNN(Convolutional Neural Network) で分析する

今まで、MNIST や Fasion MNIST といった有名なデータセットを使ってきたが、研究や趣味などで自分の好きなデータセットを分類したい場合は多い。

今回は、自作データセットを読み込み CNN で分類を行う。

## `OpenCV` の使い方

`OpenCV` は BSDライセンスの画像処理ライブラリ。Python と C++ で使える。

`OpenCV` 以外にも `Pillow` や `Numpy` も画像処理に使う。

標準画像データベースSIDBA(Standard Image Data-BAse)が公開している有名な画像を使う。

http://www.ess.ic.kanagawa-it.ac.jp/app_images_j.html よりマンドリル(サルの一種)の画像を `curl` で取ってくる。

`-o` オプションで `Mandrill.jpg` という名前にする。

`colab` ファイル内に `Mandrill.jpg` が増えている。

In [ ]:
!curl http://www.ess.ic.kanagawa-it.ac.jp/std_img/colorimage/Mandrill.jpg


`opencv 画像 表示` で検索し、画像(`Mandrill.jpg`)を**グレースケール**で表示する。

このとき、`colab` 特有のエラーが出たりする場合もあるので、その際は、エラーログを調べる。

`opencv python` で検索すると様々な機能を紹介するチュートリアルを見つけられる。

遊んでみると結構楽しい

カラー画像のヒストグラムをプロットしてみると、赤と青の輝度が大きい事が分かる

エッジ検出してみる。

`min_val` を 100、`max_val` を 200 ぐらいにするといい感じ

## 犬と猫を分類する

画像を表示してみる。

### データセットを用意する

今回は 手作業でも `curl` でも出来るようにしている。


#### 手作業の場合

下記の URL から zip ファイルをダウンロードし、Google ドライブにファイルをアップロードする。

https://www.microsoft.com/en-us/download/details.aspx?id=54765

`drive.mount('/content/drive')` でドライブをマウントし、`MyDrive`内にある zip ファイルを `unzip` する。

#### curl の場合

`-o` オプションで `cat_dog.zip` という名前にする。

あとは、`unzip`すれば良い。

In [ ]:
!curl https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

### データセットを確認する

画像の命名規則は `数字` + `.jpg` になっていて、サイズは不揃い。

In [ ]:
from matplotlib import pyplot
from matplotlib.image import imread

folder = "/content/PetImages/Dog/"

for i in range(9):
	pyplot.subplot(330 + 1 + i)
	filename = folder + str(i) + '.jpg'
  # 読み込み

  # 表示
  
pyplot.show()

自作データセットでは、画像サイズが揃っていない場合が多いため、全て揃えてやる必要がある。

拡張子が揃っていない場合、pillow で変換してやると良い。

In [ ]:
from matplotlib import pyplot
from matplotlib.image import imread
folder = "/content/PetImages/Cat/"

for i in range(9):
	pyplot.subplot(330 + 1 + i)
	filename = folder + str(i) + '.jpg'
	# 読み込み

  # 表示

pyplot.show()

In [ ]:
import matplotlib.pyplot as plt
import os
import cv2
import random
import numpy as np
from tqdm.notebook import tqdm


DATADIR =  # 全データセットが含まれるフォルダ名

CATEGORIES = [] # 各フォルダ名(カテゴリ名)を指定

IMG_SIZE = 50 # 画像サイズを指定

training_data = [] # 画像データ、ラベル情報 を入れるための空配列


def create_training_data():
    # enumerate() を使うと　インデックス と 要素 を同時に取得できる
    for class_num, category in enumerate(CATEGORIES):
        path = os.path.join(DATADIR, category)
        for image_name in tqdm(os.listdir(path)):
            try:
                # 画像読み込み ファイル名は、path と image_name をつなげる

                # 画像のリサイズ

                # 画像データ、ラベル情報を配列に追加

            except Exception as e:
                pass

create_training_data()


random.shuffle(training_data)  # データをシャッフル


X = []  # 画像データ
y = []  # ラベル情報

# データセット作成
# 画像(feature), ラベル情報(label)
for feature, label in training_data:
    X.append(feature)
    y.append(label)

# numpy配列に変換
X = np.array(X)
y = np.array(y)

ラベルとデータを可視化

In [ ]:
# データセットの確認
for i in range(0, 4):
    print("学習データのラベル：", y[i])
    plt.subplot(2, 2, i+1)
    plt.axis('off')
    plt.title(label = 'Dog' if y[i] == 0 else 'Cat')
    img_array = cv2.cvtColor(X[i], cv2.COLOR_BGR2RGB)
    plt.imshow(img_array)

plt.show()

`shape` で X と y の次元数を確認する

50 x 50 の RGB であることがわかる

### ホールドアウト法でデータセットを分割

train, test を 8:2 で分割

X_train, X_test, y_train, y_test の次元を確認する

`BGR` になっているので `RGB` に変更する。

In [ ]:
fig = plt.figure(figsize=(9, 15))
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.5, hspace=0.05,
                    wspace=0.05)

for i in range(9):
    ax = fig.add_subplot(1, 9, i + 1, xticks=[], yticks=[])
    img_array = cv2.cvtColor(X_train[i], cv2.COLOR_BGR2RGB)
    ax.imshow(img_array)

`X_train`, `X_test` は画素値を `255` で割ることで `0` ~ `1` に正規化する。配列を形状変換する。

`y_train`, `y_test` は正解ラベルを One-hot エンコーディングする。

In [ ]:
from tensorflow.keras.utils import to_categorical

X_train = X_train.reshape((X_train.shape[0], ?, ?, ?)) / ???
X_test = X_test.reshape((X_test.shape[0], ?, ?, ?)) / ???


X_train, X_test, y_train, y_test の次元を確認すると変わっている点がある。

### モデル定義

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
%load_ext tensorboard

**好きなアーキテクチャーの CNN を実装する。**

損失関数 : バイナリクロスエントロピー

最適化手法 : Adam

metrics : accuracy

モデルを可視化する

In [ ]:
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

Keras の実装は下記 URL をみると様々な実装があって良い。

https://keras.io/examples

### 学習

In [ ]:
tf_callback = TensorBoard(log_dir="logs", histogram_freq=1)

hist = model.fit(, batch_size=128, epochs=10, verbose=1, , callbacks=[tf_callback])

### 検証

TensorBoard で結果を可視化する。

In [ ]:
%tensorboard --logdir logs

In [ ]:
plt.figure(figsize = (18,6))

# accuracy
plt.subplot(1, 2, 1)
plt.plot(hist.history["accuracy"], label = "acc", marker = "o")
plt.plot(hist.history["val_accuracy"], label = "val_acc", marker = "o")

# x ラベルを epoch

# y ラベルを accuracy

plt.legend(loc = "best")
plt.grid(color = 'gray', alpha = 0.2)

# loss
plt.subplot(1, 2, 2)
plt.plot(hist.history["loss"], label = "loss", marker = "o")
plt.plot(hist.history["val_loss"], label = "val_loss", marker = "o")

plt.xlabel("epoch")
plt.ylabel("loss")


plt.legend(loc = "best")

#　グリッド。透明度 0.2
plt.grid(color = 'gray', alpha = 0.2)


検証データの損失と精度を求める

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)
print("evaluate loss: {0[0]}".format(score))
print("evaluate acc: {0[1]}".format(score))

### モデルを保存

HDF5 形式で保存する。

In [ ]:
model.save('Cat_And_Dog_Net.h5')

### 学習済みモデルを使って予測する

今回の場合、学習から行っているためモデルを保存する必要はないが、保存することで学習済みデータさえあれば予測をどこでもできるようになる。

In [ ]:
model = load_model('Cat_And_Dog_Net.h5')

In [ ]:
labels = np.array([
   'Dog','Cat'
])

テストデータと正解ラベルを表示

In [ ]:
true_classes = np.argmax(y_test[0:30], axis = 1)

# testデータ30件の画像と正解ラベルを出力
plt.figure(figsize = (16, 6))
for i in range(30):
    plt.subplot(3, 10, i + 1)
    plt.axis("off")
    plt.title(labels[true_classes[i]])
    plt.imshow(X_test[i])
plt.show()

In [ ]:
pred_classes = np.argmax(model.predict(X_test[0:30]), axis=-1)

# testデータ30件の予測確率
pred_probs = model.predict(X_test[0:30]).max(axis = 1)
pred_probs = ['{:.4f}'.format(i) for i in pred_probs]

# testデータ30件の画像と予測ラベル＆予測確率を出力
plt.figure(figsize = (16, 6))
for i in range(30):
    plt.subplot(3, 10, i + 1)
    plt.axis("off")
    if pred_classes[i] == true_classes[i]:
        plt.title(labels[pred_classes[i]] + '\n' + pred_probs[i])
    else:
        plt.title(labels[pred_classes[i]] + '\n' + pred_probs[i], color = "red")
    plt.imshow(X_test[i])
plt.show()

#### 外部から持って来た画像を予測する

In [ ]:
from PIL import Image
import glob

image_paths = glob.glob('img/*.jpg')
image_paths

In [ ]:
# 画像を中央でクロップ
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop((
        (img_width - crop_width) // 2,
        (img_height - crop_height) // 2,
        (img_width + crop_width) // 2,
        (img_height + crop_height) // 2
    ))

# 画像リサイズ
def crop_resize(image_path, resize_width, resize_height):
    image = Image.open(image_path)
    crop = crop_center(image, min(image.size), min(image.size))
    resized = crop.resize((resize_width, resize_height))
    img = np.array(resized).astype("float32")
    img /= 255
    return img

images = [crop_resize(p, 50, 50) for p in image_paths]
# 配列に変換
images = np.asarray(images)

In [ ]:
true_classes = np.array([list(labels).index('Dog'),list(labels).index('Cat')])
print(true_classes)

In [ ]:
# 外部画像の予測ラベル
pred_classes = np.argmax(model.predict(images), axis=-1)

# 外部画像の予測確率
pred_probs = model.predict(images).max(axis = 1)
pred_probs = ['{:.4f}'.format(i) for i in pred_probs]

# 外部画像と予測ラベル&予測確率を出力
plt.figure(figsize = (16, 6))
for i in range(2):
    plt.subplot(1, 10, i + 1)
    plt.axis("off")
    if pred_classes[i] == true_classes[i]:
        plt.title(labels[pred_classes[i]] + '\n' + pred_probs[i])
    else:
        plt.title(labels[pred_classes[i]] + '\n' + pred_probs[i], color = "red")
    plt.imshow(images[i])
plt.show()